In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv("musteri_kaybi_modellemesi.csv")
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
''' RowNumber, CustomerId ve Surname'in
    sonuca bir etkisi olmayacağı için 
    eğitimde kullanmıyoruz.'''

X = dataset.iloc[:,3:13].values
y = dataset.iloc[:,13].values

In [4]:
# Kategorik Veriler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

# Geography
print("LabelEncoder Uygulamadan onceki X degerleri: \n")
print(X)
print ("\n")
labelEncoder_X1 = LabelEncoder()
X[:,1] = labelEncoder_X1.fit_transform(X[:,1])

# Gender
labelEncoder_X2 = LabelEncoder()
X[:,2] = labelEncoder_X2.fit_transform(X[:,2])
print("LabelEncoder Uygulandıktan sonraki X degerleri: \n")
print(X)
print ("\n")

ohe = OneHotEncoder(categorical_features = [1])
X = ohe.fit_transform(X).toarray()
# Dummy Variable tuzağından kaçınmak için 1 değeri  drop ediyoruz.
X = X[:,1:]

print("OneHotEncoder Uygulandıktan sonraki X[1] degerleri: \n ")
print(X)
print ("\n")



LabelEncoder Uygulamadan onceki X degerleri: 

[[619 'France' 'Female' ..., 1 1 101348.88]
 [608 'Spain' 'Female' ..., 0 1 112542.58]
 [502 'France' 'Female' ..., 1 0 113931.57]
 ..., 
 [709 'France' 'Female' ..., 0 1 42085.58]
 [772 'Germany' 'Male' ..., 1 0 92888.52]
 [792 'France' 'Female' ..., 1 0 38190.78]]


LabelEncoder Uygulandıktan sonraki X degerleri: 

[[619 0 0 ..., 1 1 101348.88]
 [608 2 0 ..., 0 1 112542.58]
 [502 0 0 ..., 1 0 113931.57]
 ..., 
 [709 0 0 ..., 0 1 42085.58]
 [772 1 1 ..., 1 0 92888.52]
 [792 0 0 ..., 1 0 38190.78]]


OneHotEncoder Uygulandıktan sonraki X[1] degerleri: 
 
[[  0.00000000e+00   0.00000000e+00   6.19000000e+02 ...,   1.00000000e+00
    1.00000000e+00   1.01348880e+05]
 [  0.00000000e+00   1.00000000e+00   6.08000000e+02 ...,   0.00000000e+00
    1.00000000e+00   1.12542580e+05]
 [  0.00000000e+00   0.00000000e+00   5.02000000e+02 ...,   1.00000000e+00
    0.00000000e+00   1.13931570e+05]
 ..., 
 [  0.00000000e+00   0.00000000e+00   7.09000000e

In [5]:
# Eğitim ve Veri Seti Olarak Ayırma 

from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

In [6]:
# Verileri Belirli Bir Aralığa Sokma

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train) 
X_test = sc.transform(X_test)

In [7]:
# Part 2 - ANN!

from keras.models import Sequential # ANN'i baslatir
from keras.layers import Dense # Dense layer olusturmak icin kullanılır

Using TensorFlow backend.


In [8]:
# ANN'i baslatma
classifier = Sequential()

In [9]:
# İlk katmanın oluşturulması
# Units: Node sayısı. Input sayısı + Output Sayısının yarısı. 
# kernel_initializer: Ağırlıklara random değerler atar.
# Activation: Relu

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

In [10]:
# Gizli katmanın eklenmesi
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

In [11]:
# Çıktı katmanını ekleme
# Units: Müsteri bankayı terk edecek mi etmeyecek mi? Binary Outcome olduğu için 1 node'a ihtiyaç var.
# Activation: Sigmoid -> Binary outcome' lar için kullanılır.
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [12]:
# ANN'i derleme
# Optimizer: Gradient Descent -> Adam
# Loss: Binary outcome olduğu için binary_crossentropy olarak seçtik. Fakat çıktı değerimiz 2 den fazla olsaydı
#       categorical_crossentropy seçecektik.
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [13]:
# ANN Eğitim
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s - loss: 0.4878 - acc: 0.7954     
Epoch 2/100
8000/8000 [==============================] - 1s - loss: 0.4280 - acc: 0.7960     
Epoch 3/100
8000/8000 [==============================] - 1s - loss: 0.4231 - acc: 0.7969     
Epoch 4/100
8000/8000 [==============================] - 1s - loss: 0.4190 - acc: 0.8209     
Epoch 5/100
8000/8000 [==============================] - 1s - loss: 0.4165 - acc: 0.8250     
Epoch 6/100
8000/8000 [==============================] - 1s - loss: 0.4145 - acc: 0.8282     
Epoch 7/100
8000/8000 [==============================] - 1s - loss: 0.4131 - acc: 0.8300     
Epoch 8/100
8000/8000 [==============================] - 1s - loss: 0.4116 - acc: 0.8321     
Epoch 9/100
8000/8000 [==============================] - 1s - loss: 0.4104 - acc: 0.8327     
Epoch 10/100
8000/8000 [==============================] - 1s - loss: 0.4093 - acc: 0.8339     
Epoch 11/100
8000/8000 [==============================] - 1

8000/8000 [==============================] - 1s - loss: 0.3993 - acc: 0.8359     
Epoch 88/100
8000/8000 [==============================] - 1s - loss: 0.3998 - acc: 0.8369     
Epoch 89/100
8000/8000 [==============================] - 1s - loss: 0.3994 - acc: 0.8369     
Epoch 90/100
8000/8000 [==============================] - 1s - loss: 0.3998 - acc: 0.8349     
Epoch 91/100
8000/8000 [==============================] - 1s - loss: 0.3991 - acc: 0.8350     
Epoch 92/100
8000/8000 [==============================] - 1s - loss: 0.3994 - acc: 0.8357     
Epoch 93/100
8000/8000 [==============================] - 1s - loss: 0.3997 - acc: 0.8346     
Epoch 94/100
8000/8000 [==============================] - 1s - loss: 0.3995 - acc: 0.8365     
Epoch 95/100
8000/8000 [==============================] - 1s - loss: 0.3992 - acc: 0.8362     
Epoch 96/100
8000/8000 [==============================] - 1s - loss: 0.3990 - acc: 0.8355     
Epoch 97/100
8000/8000 [==============================] - 1s - 

In [14]:
# Part 3 - Tahmin yapma ve modelin değerlendirilmesi
# Test seti sonuçlarının tahmini

y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
print("Test verisi sonuclari: \n")
y_pred

Test verisi sonuclari: 



array([[False],
       [False],
       [False],
       ..., 
       [False],
       [False],
       [False]], dtype=bool)

In [15]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print("\n Confusion Matrix: \n")
print(cm)


 Confusion Matrix: 

[[1556   39]
 [ 282  123]]


In [16]:
"""Asagidaki bilgilere sahip musteri bankayı terk edecek mi?:
Geography: France
Credit Score: 600
Gender: Female
Age: 45
Tenure: 3
Balance: 75000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 60000"""

predict_customer = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 45, 3, 75000, 2, 1, 1, 60000]])))
predict_customer = (predict_customer > 0.5)

predict_customer

array([[False]], dtype=bool)

In [17]:
# Part 4 - Evaluating, Improving and Tuning the ANN

# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [18]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [19]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)

In [ ]:
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs=1)

Epoch 1/100
7200/7200 [==============================] - 1s - loss: 0.5042 - acc: 0.7960     
Epoch 2/100
7200/7200 [==============================] - 1s - loss: 0.4298 - acc: 0.7971     
Epoch 3/100
7200/7200 [==============================] - 1s - loss: 0.4261 - acc: 0.7971     
Epoch 4/100
7200/7200 [==============================] - 1s - loss: 0.4223 - acc: 0.7971     
Epoch 5/100
7200/7200 [==============================] - 1s - loss: 0.4201 - acc: 0.8132     
Epoch 6/100
7200/7200 [==============================] - 1s - loss: 0.4177 - acc: 0.8204     
Epoch 7/100
7200/7200 [==============================] - 1s - loss: 0.4160 - acc: 0.8269     
Epoch 8/100
7200/7200 [==============================] - 1s - loss: 0.4145 - acc: 0.8281     
Epoch 9/100
7200/7200 [==============================] - 1s - loss: 0.4130 - acc: 0.8314     
Epoch 10/100
7200/7200 [==============================] - 1s - loss: 0.4112 - acc: 0.8317     
Epoch 11/100
7200/7200 [==============================] - 1

In [ ]:
mean = accuracies.mean()
variance = accuracies.std()
print(mean)
print(variance)

In [ ]:
# Doğruluk artırma
# Dropout Regularization: Overfitting i azaltmak için

# Tuning 
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [ ]:
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}

In [ ]:
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)

In [ ]:
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

print("\n Best Parameters: \n")
print(best_parameters)
print("\n Best Accuracy: \n")
print(best_accuracy)